In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("proj2.ipynb")

# Project 2: Query Performance
## Due Date: Friday 10/07, 11:59 PM
## Assignment Details
In this project, we will explore how the database system optimizes query execution and how users can futher tune the performance of their queries.

This project works with the Lahman's Baseball Database, an open source collection of baseball statistics from 1871 to 2020. It contains a variety of data, like batting statistics, team stats, managerial records, Hall of Fame records, and much more.

**Note:** If at any point during the project, the internal state of the database or its tables have been modified in an undesirable way (i.e. a modification not resulting from the instructions of a question), restart your kernel and clear output and simply re-run the notebook as normal. This will shutdown you current connection to the database, which will prevent the issue of multiple connections to the database at any given point, and when re-running the notebook you will create a fresh database based on the provided Postgres dump.

## Logistics & Scoring Breakdown

For Data 101 students, this project is worth 15% of your grade. For Info 258 students, this project is worth 12% of your grade.

- Each coding question has **both public tests and hidden tests**. Roughly 50% of your coding grade will be made up of your score on the public tests released to you, while the remaining 50% will be made up of unreleased hidden tests.
- Public tests for multiple choice questions are for sanity check only (e.g. you are answering in the correct format). Partial credit will be awarded.
- Free-response questions will be manually graded. Please answer thoughtfully in complete sentences, drawing from knowledge in lectures and from your inspection of query plans.

This is an **individual project**. However, you’re welcome to collaborate with any other student in the class as long as it’s within the academic honesty guidelines.

Question | Points
--- | ---
0 | 1
1a	| 1
1bi	| 1
1bii	| 2
1c	| 1
1di	| 2
1dii	| 1
1ei	| 1
1eii	| 1
1eiii	| 1
1eiv	| 2
1ev	| 2
2a	| 1
2b	| 1
2c	| 2
2d	| 2
3a	| 1
3b	| 1
3c |	1
3d	| 3
4a	| 2
4b	| 1
4c	| 1
4di	| 2
4dii |	2
4ei	| 2
4eii | 	1
4eiii |	2
4eiv |	1
4ev |	1
4evi |	2
4evii |	2
5a | 2
5b | 2
5c | 1
5d | 2
6a | 2
6b | 1
6c | 1
6d | 2
6e | 2
7a | 1
7b | 1
7c | 2
8  | 6
**Total** | 72

In [2]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

## Getting Connected
Similar to Project 1, we will be using the `ipython-sql` library to connect this notebook to a PostgreSQL database server on your JupyterHub account. Run the following cell to initiate the connection.

In [3]:
%reload_ext sql
%sql postgresql://jovyan@127.0.0.1:5432/postgres

## Setting up the Database
The following cell will create the `baseball` database (if needed), unzip the Postgres dump of the Lahman's Baseball Database, populate the `baseball` database with the desired tables and data, and finally display all databases associated with the Postgres instance. After running the cell, you should see the `baseball` database in the generated list of databases outputted by `%sql \l`.

**Note:** If you run into the **role does not exist** error, feel free to ignore it. It does not affect data import.

In [4]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS baseball'
!psql -h localhost -c 'CREATE DATABASE baseball'
!gzip -dc data/lahman.pgdump.gz | psql -h localhost -d baseball -f -
!psql -h localhost -c 'SET max_parallel_workers_per_gather = 0;'
%sql \l

DROP DATABASE
CREATE DATABASE
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
CREATE EXTENSION
COMMENT
SET
CREATE TABLE
psql:<stdin>:49: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:80: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:96: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:112: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:129: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:146: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:178: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:210: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:223: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:251: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:267: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:295: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:322: ERROR:  

Name,Owner,Encoding,Collate,Ctype,Access privileges
baseball,jovyan,UTF8,en_US.utf8,en_US.utf8,None
imdb,jovyan,UTF8,en_US.utf8,en_US.utf8,None
jovyan,jovyan,UTF8,en_US.utf8,en_US.utf8,None
postgres,jovyan,UTF8,en_US.utf8,en_US.utf8,None
template0,jovyan,UTF8,en_US.utf8,en_US.utf8,=c/jovyanjovyan=CTc/jovyan
template1,jovyan,UTF8,en_US.utf8,en_US.utf8,=c/jovyanjovyan=CTc/jovyan


Now, run the following cell to connect to the `baseball` database. There should be no errors after running the following cell.

In [5]:
%sql postgresql://jovyan@127.0.0.1:5432/baseball

To ensure that the connection to the database has been established, let's try grabbing the first 5 rows from the `halloffame` table.

In [6]:
%%sql
SELECT * FROM halloffame LIMIT 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


playerid,yearid,votedby,ballots,needed,votes,inducted,category,needed_note
cobbty01,1936,BBWAA,226,170,222,Y,Player,None
ruthba01,1936,BBWAA,226,170,215,Y,Player,None
wagneho01,1936,BBWAA,226,170,215,Y,Player,None
mathech01,1936,BBWAA,226,170,205,Y,Player,None
johnswa01,1936,BBWAA,226,170,189,Y,Player,None


Run the following cell for grading purposes.

In [7]:
!mkdir -p results

## Table Descriptions
In its entirety the Lahman's Baseball Database contains 27 tables containing a variety of statistics for players, teams, games, schools, etc. For simplicity, this project will focus on a subset of the tables:
* `appearances`: details on the positions each player appeared at
* `batting`: batting statistics for each player
* `collegeplaying`: list of players and the colleges they attended
* `halloffame`: Hall of Fame voting data
* `people`: player information (name, date of birth, and biographical info)
* `salaries`: player salary data
* `schools`: list of colleges that players attended

As a reminder from Project 1, `%sql \d <table_name>` is helpful for identifying the columns in a table.

## Question 0: PostgreSQL Explain Analyze
**Please read through this section carefully, as a vast majority of the project will require you to inspect query plans via interpreting the output of the explain analyze command.**

To inspect the query plan for a given query, create a variable storing the query as a string and invoke a `psql` shell command to `explain analyze` the query: 

`your_variable = "__REPLACE_ME_WITH_QUERY__"`

`!psql -h localhost -d baseball -c "explain analyze $your_variable"`

Take a look at the following sample query plan.

![title](data/sample_query.png)

It is highly recommended to read through [this article](https://www.cybertec-postgresql.com/en/how-to-interpret-postgresql-explain-analyze-output/) to see how you can interpret the output above. Everything before "Tools to interpret Explain Analyze output" is useful.

Here are some key things to note:
- There are two cost values: the first is the **startup cost** (cost to return the first row) and the second is the **total cost** (cost to return all rows).
- The unit for the estimated query cost is arbitrary (1 is the cost for reading an 8kB page during a sequential scan)
- When we ask you to identify the query cost, we are looking for the **total cost**
- When we ask you to identify the query time, we are looking for the **execution time** (in ms)
- Feel free to round the query cost / time to the nearest integer, but we'll accept anything more exact
- We recognzie that the execution time may vary between different cell executions, so the autograder will tolerate a reasonable range

Now, inspect the query plan above, and record the **cost** and **time** for the sample query.

<!--
BEGIN QUESTION
name: q0
points: 1
-->

In [8]:
sample_query_cost = 1193.88
sample_query_timing = 185.896

In [9]:
grader.check("q0")

q0 results: All test cases passed!

## Question 1: Queries and Views
### Question 1a:
Write a query that finds `namefirst`, `namelast`, `playerid` and `yearid` of all people who were successfully inducted into the Hall of Fame. **Note**: Your query should **NOT** use any sub-queries.

In [10]:
%%sql result_1a <<
SELECT p.namefirst, p.namelast, h.yearid, h.playerid
FROM halloffame as h
LEFT JOIN people as p
ON h.playerid = p.playerid
WHERE inducted = 'Y'

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
323 rows affected.
Returning data to local variable result_1a


In [11]:
# Do not delete/edit this cell
result_1a.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_1a.csv', index=False)

In [12]:
grader.check("q1a")

q1a results: All test cases passed!

### Question 1b:
In this question, we will compare the query you wrote in `Question 1a` against the provided query in `Question 1bi` by inspecting the query plans.

#### Question 1bi: 
Inspect the query plan for `provided_query` and the query you wrote in `Question 1a`. Record the **execution time** and **cost** for each query.

In [13]:
%%sql provided_query <<
SELECT namefirst, namelast, p.playerid, yearid
FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof 
WHERE p.playerid = hof.playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
323 rows affected.
Returning data to local variable provided_query


In [14]:
provided_query_str = "SELECT namefirst, namelast, p.playerid, yearid FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof WHERE hof.inducted = 'Y' AND p.playerid = hof.playerid;"
!psql -h localhost -d baseball -c "explain analyze $provided_query_str"

                                                       QUERY PLAN                                                        
-------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=861.83..959.06 rows=323 width=25) (actual time=10.313..10.872 rows=323 loops=1)
   Hash Cond: ((halloffame.playerid)::text = (p.playerid)::text)
   ->  Seq Scan on halloffame  (cost=0.00..96.39 rows=323 width=13) (actual time=0.023..0.492 rows=323 loops=1)
         Filter: ((inducted)::text = 'Y'::text)
         Rows Removed by Filter: 3868
   ->  Hash  (cost=619.70..619.70 rows=19370 width=21) (actual time=10.118..10.118 rows=19370 loops=1)
         Buckets: 32768  Batches: 1  Memory Usage: 1293kB
         ->  Seq Scan on people p  (cost=0.00..619.70 rows=19370 width=21) (actual time=0.008..5.069 rows=19370 loops=1)
 Planning Time: 0.818 ms
 Execution Time: 10.974 ms
(10 rows)



In [15]:
inducted_hof_query_str = "SELECT p.namefirst, p.namelast, h.yearid, h.playerid FROM halloffame as h LEFT JOIN people as p ON h.playerid = p.playerid WHERE inducted = 'Y'"
!psql -h localhost -d baseball -c "explain analyze $inducted_hof_query_str"

                                                       QUERY PLAN                                                        
-------------------------------------------------------------------------------------------------------------------------
 Hash Left Join  (cost=861.83..959.06 rows=323 width=25) (actual time=9.636..10.247 rows=323 loops=1)
   Hash Cond: ((h.playerid)::text = (p.playerid)::text)
   ->  Seq Scan on halloffame h  (cost=0.00..96.39 rows=323 width=13) (actual time=0.021..0.536 rows=323 loops=1)
         Filter: ((inducted)::text = 'Y'::text)
         Rows Removed by Filter: 3868
   ->  Hash  (cost=619.70..619.70 rows=19370 width=21) (actual time=9.437..9.437 rows=19370 loops=1)
         Buckets: 32768  Batches: 1  Memory Usage: 1293kB
         ->  Seq Scan on people p  (cost=0.00..619.70 rows=19370 width=21) (actual time=0.009..4.781 rows=19370 loops=1)
 Planning Time: 0.858 ms
 Execution Time: 10.321 ms
(10 rows)



In [16]:
provided_query_cost = 959.06
provided_query_timing = 11.657
your_query_cost = 959.06
your_query_timing = 11.945

In [17]:
grader.check("q1bi")

q1bi results: All test cases passed!

#### Question 1bii:
Given your findings from inspecting the query plans of the two queries, answer the following question. Assign the variable `q1b_part2` to a list of answer choices which are true statements.

1. Consider the following statements:
    1. Both the queries have the same cost.
    1. The provided query has a faster execution time because it makes use of a subquery.
    1. The query you wrote has a faster execution time because it does not make use a subquery.
    1. The provided query has less cost because it makes use of a subquery.
    1. The query you wrote has less cost because it does not make use a subquery.
    1. The queries have the same output.
    1. The queries do not have the same output.
    
**Note:** Your answer should look like `q1b_part2 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q1bii
points: 2
-->

In [18]:
q1b_part2 = ['A', 'C', 'F']

In [19]:
grader.check("q1bii")

q1bii results: All test cases passed!

### Question 1c:
Write a query that creates a view named `inducted_hof_ca` of the people who were successfully inducted into the Hall of Fame and played in college at a school located in California. For each player, return their `namefirst`, `namelast`, `playerid`, `schoolid`, and `yearid` ordered by the `yearid` and then the `playerid`. 

**Note**: For this query, `yearid` refers to player's year of induction into the Hall of Fame.

In [20]:
%%sql result_1c <<
DROP VIEW IF EXISTS inducted_hof_ca;
CREATE VIEW inducted_hof_ca AS
SELECT hp.namefirst, hp.namelast, hp.playerid, cs.schoolid, hp.yearid
FROM (SELECT p.namefirst, p.namelast, h.yearid, h.playerid FROM halloffame as h LEFT JOIN people as p ON h.playerid = p.playerid WHERE inducted = 'Y') as hp
JOIN (SELECT c.playerid, c.schoolid, s.schoolstate FROM collegeplaying AS c JOIN schools AS s ON c.schoolid = s.schoolid WHERE schoolstate = 'CA') AS cs
ON hp.playerid = cs.playerid
ORDER BY yearid, playerid
;
SELECT * FROM inducted_hof_ca;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.
23 rows affected.
Returning data to local variable result_1c


In [21]:
# Do not delete/edit this cell
result_1c.DataFrame().sort_values(['playerid', 'schoolid', 'yearid']).to_csv('results/result_1c.csv', index=False)

In [22]:
grader.check("q1c")

q1c results: All test cases passed!

### Question 1d:
For this question, we want to compute the count of players who were inducted into the Hall of Fame and played baseball at a college in California for each `schoolid` and `yearid` combination ordered by ascending `yearid`.

#### Question 1di:
Write two queries that accomplish this task -- one query using the view you created in the `Question 1c` and one query that does not use the view, common table expressions (CTEs), or any sub-queries.

In [23]:
%%sql with_view <<
SELECT schoolid, yearid, COUNT(*)
FROM inducted_hof_ca
GROUP BY schoolid, yearid
ORDER BY yearid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable with_view


In [24]:
%%sql without_view <<
SELECT s.schoolid, h.yearid, COUNT(*)
FROM people AS p
LEFT JOIN halloffame AS h ON p.playerid = h.playerid
LEFT JOIN collegeplaying AS c ON h.playerid = c.playerid
LEFT JOIN schools AS s ON c.schoolid = s.schoolid
WHERE h.inducted = 'Y' AND s.schoolstate = 'CA'
GROUP BY s.schoolid, h.yearid
ORDER BY h.yearid

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable without_view


In [25]:
# Do not delete/edit this cell
with_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1di_view.csv', index=False)
without_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1di_no_view.csv', index=False)

In [26]:
grader.check("q1di")

q1di results: All test cases passed!

#### Question 1dii:
Fill in your queries from `Question 1di` and inspect the query plans for the two queries. Record the execution time and cost for each query.

In [27]:
with_view_query_str = "SELECT schoolid, yearid, COUNT(*) FROM inducted_hof_ca GROUP BY schoolid, yearid ORDER BY yearid"
!psql -h localhost -d baseball -c "explain analyze $with_view_query_str"

                                                                          QUERY PLAN                                                                           
---------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=493.31..495.23 rows=96 width=20) (actual time=5.544..5.553 rows=13 loops=1)
   Group Key: inducted_hof_ca.yearid, inducted_hof_ca.schoolid
   ->  Sort  (cost=493.31..493.55 rows=96 width=12) (actual time=5.539..5.543 rows=23 loops=1)
         Sort Key: inducted_hof_ca.yearid, inducted_hof_ca.schoolid
         Sort Method: quicksort  Memory: 26kB
         ->  Subquery Scan on inducted_hof_ca  (cost=488.95..490.15 rows=96 width=12) (actual time=5.483..5.489 rows=23 loops=1)
               ->  Sort  (cost=488.95..489.19 rows=96 width=257) (actual time=5.482..5.485 rows=23 loops=1)
                     Sort Key: h.yearid, h.playerid
                    

In [28]:
without_view_query_str = "SELECT s.schoolid, h.yearid, COUNT(*) FROM people AS p LEFT JOIN halloffame AS h ON p.playerid = h.playerid LEFT JOIN collegeplaying AS c ON h.playerid = c.playerid LEFT JOIN schools AS s ON c.schoolid = s.schoolid WHERE h.inducted = 'Y' AND s.schoolstate = 'CA' GROUP BY s.schoolid, h.yearid ORDER BY h.yearid"
!psql -h localhost -d baseball -c"explain analyze $without_view_query_str"

                                                                       QUERY PLAN                                                                        
---------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=518.79..519.45 rows=33 width=20) (actual time=6.999..7.015 rows=13 loops=1)
   Group Key: h.yearid, s.schoolid
   ->  Sort  (cost=518.79..518.87 rows=33 width=12) (actual time=6.992..6.997 rows=23 loops=1)
         Sort Key: h.yearid, s.schoolid
         Sort Method: quicksort  Memory: 26kB
         ->  Nested Loop  (cost=386.71..517.95 rows=33 width=12) (actual time=6.279..6.859 rows=23 loops=1)
               Join Filter: ((h.playerid)::text = (p.playerid)::text)
               ->  Hash Join  (cost=386.42..485.79 rows=96 width=30) (actual time=6.253..6.695 rows=23 loops=1)
                     Hash Cond: ((h.playerid)::text = (c.playerid)::text)
             

In [29]:
with_view_cost = 515.79
with_view_timing = 6.086
without_view_cost = 539.67
without_view_timing = 5.989

In [30]:
grader.check("q1dii")

q1dii results: All test cases passed!

### Question 1e:
#### Question 1ei:
Now, let's try creating a materialized view named `inducted_hof_ca_mat` instead of the regular view from `Question 1c`.

In [31]:
%%sql inducted_hof_ca_materialized <<
DROP MATERIALIZED VIEW IF EXISTS inducted_hof_ca_mat;
CREATE MATERIALIZED VIEW inducted_hof_ca_mat AS
SELECT hp.namefirst, hp.namelast, hp.playerid, cs.schoolid, hp.yearid
FROM (SELECT p.namefirst, p.namelast, h.yearid, h.playerid FROM halloffame as h LEFT JOIN people as p ON h.playerid = p.playerid WHERE inducted = 'Y') as hp
JOIN (SELECT c.playerid, c.schoolid, s.schoolstate FROM collegeplaying AS c JOIN schools AS s ON c.schoolid = s.schoolid WHERE schoolstate = 'CA') AS cs
ON hp.playerid = cs.playerid
ORDER BY yearid, playerid
;
SELECT * FROM inducted_hof_ca_mat;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
23 rows affected.
23 rows affected.
Returning data to local variable inducted_hof_ca_materialized


In [32]:
# Do not delete/edit this cell
inducted_hof_ca_materialized.DataFrame().sort_values(['playerid', 'schoolid', 'yearid']).to_csv('results/result_1ei.csv', index=False)

In [33]:
grader.check("q1ei")

q1ei results: All test cases passed!

#### Question 1eii:

Now, rewrite the query from `Question 1d` to use the materialized view `inducted_hof_ca_mat`.

In [34]:
%%sql with_materialized_view <<
SELECT schoolid, yearid, COUNT(*)
FROM inducted_hof_ca_mat
GROUP BY schoolid, yearid
ORDER BY yearid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable with_materialized_view


In [35]:
# Do not delete/edit this cell
with_materialized_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1eii.csv', index=False)

In [36]:
grader.check("q1eii")

q1eii results: All test cases passed!

#### Question 1eiii:
Inspect the query plan and record the execution time and cost of the query that uses the materialized view.

In [37]:
with_materialized_view_query_str = "SELECT schoolid, yearid, COUNT(*) FROM inducted_hof_ca_mat GROUP BY schoolid, yearid ORDER BY yearid;"
!psql -h localhost -d baseball -c "explain analyze $with_materialized_view_query_str"

                                                          QUERY PLAN                                                          
------------------------------------------------------------------------------------------------------------------------------
 Sort  (cost=23.67..24.17 rows=200 width=60) (actual time=0.047..0.049 rows=13 loops=1)
   Sort Key: yearid
   Sort Method: quicksort  Memory: 25kB
   ->  HashAggregate  (cost=14.03..16.02 rows=200 width=60) (actual time=0.022..0.026 rows=13 loops=1)
         Group Key: yearid, schoolid
         Batches: 1  Memory Usage: 40kB
         ->  Seq Scan on inducted_hof_ca_mat  (cost=0.00..12.30 rows=230 width=52) (actual time=0.005..0.007 rows=23 loops=1)
 Planning Time: 0.242 ms
 Execution Time: 0.133 ms
(9 rows)



In [38]:
with_materialized_view_cost = 24.17
with_materialized_view_timing = 0.253

In [39]:
grader.check("q1eiii")

q1eiii results: All test cases passed!

#### Question 1eiv:
Given your findings from inspecting the query plans, as well as your understanding of views and materialized views from lectures, answer the following question. Assign the variable `q1e_part4` to a list of all statements which are true.

1. Consider the following statements:
    1. Views will reduce the execution time and the cost of a query.
    1. Views will reduce the execution time of a query, but not the cost.
    1. Views will reduce the cost of a query, but not the execution time.
    1. Materialized views reduce the execution time and the cost of a query.
    1. Materialized views reduce the execution time, but not cost of a query.
    1. Materialized views reduce the cost of a query, but not the execution time.
    1. Materialized views will result in the same query plan as a query using views.
    1. Materialized views and views take the same time to create.
    1. Materialized views take less time to create than a view.
    1. Materialized views take more time to create than a view.
    
*Note:* Your answer should look like `q1e_part4 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q1eiv
points: 2
-->

In [40]:
q1e_part4 = ['C', 'D', 'I']

In [41]:
grader.check("q1eiv")

q1eiv results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 1ev:

Explain your answer to the previous part based on your knowledge from lectures, and details from the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q1ev
manual: true
points: 2
-->

Views still query the actual table every time while materialized views store results on disk.

<!-- END QUESTION -->



## Question 2: Predicate Pushdown
In this question, we will explore the impact of predicates on a query's execution, particularly inspecting when the optimizer applies predicates.

### Question 2a:
Recall the `inducted_hof_ca` created in `Question 1c`. Inspect the query plan for a query that that gets all rows from the view, and record the execution time and cost.

In [42]:
query_view_str = "SELECT * FROM inducted_hof_ca"
!psql -h localhost -d baseball -c "explain analyze $query_view_str"

                                                                    QUERY PLAN                                                                     
---------------------------------------------------------------------------------------------------------------------------------------------------
 Sort  (cost=887.97..888.21 rows=96 width=33) (actual time=5.303..5.307 rows=23 loops=1)
   Sort Key: h.yearid, h.playerid
   Sort Method: quicksort  Memory: 26kB
   ->  Nested Loop Left Join  (cost=386.71..884.80 rows=96 width=33) (actual time=4.814..5.248 rows=23 loops=1)
         ->  Hash Join  (cost=386.42..485.79 rows=96 width=21) (actual time=4.790..5.093 rows=23 loops=1)
               Hash Cond: ((h.playerid)::text = (c.playerid)::text)
               ->  Seq Scan on halloffame h  (cost=0.00..96.39 rows=323 width=13) (actual time=0.021..0.523 rows=323 loops=1)
                     Filter: ((inducted)::text = 'Y'::text)
                     Rows Removed by Filter: 3868
               ->  

In [43]:
query_view_cost = 905.10
query_view_timing = 206.352

In [44]:
grader.check("q2a")

q2a results: All test cases passed!

### Question 2b:
Now, add a filter to only return rows from `inducted_hof_ca` where the year is later than 2010. Inspect the query plan and record the execution time and cost.

In [45]:
%%sql result_2b <<
SELECT *
FROM inducted_hof_ca
WHERE yearid > 2010

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
8 rows affected.
Returning data to local variable result_2b


In [46]:
query_view_with_filter_str = "SELECT * FROM inducted_hof_ca WHERE yearid > 2010"
!psql -h localhost -d baseball -c "explain analyze $query_view_with_filter_str"

                                                                             QUERY PLAN                                                                              
---------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Sort  (cost=254.86..254.88 rows=6 width=33) (actual time=0.750..0.752 rows=8 loops=1)
   Sort Key: h.yearid, h.playerid
   Sort Method: quicksort  Memory: 25kB
   ->  Nested Loop Left Join  (cost=0.85..254.78 rows=6 width=33) (actual time=0.438..0.719 rows=8 loops=1)
         ->  Nested Loop  (cost=0.56..207.24 rows=6 width=21) (actual time=0.410..0.654 rows=8 loops=1)
               ->  Nested Loop  (cost=0.29..190.63 rows=55 width=21) (actual time=0.399..0.567 rows=27 loops=1)
                     ->  Seq Scan on halloffame h  (cost=0.00..106.86 rows=21 width=13) (actual time=0.368..0.403 rows=31 loops=1)
                           Filter: ((yearid > 2010) 

In [47]:
query_view_with_filter_cost = 254.88
query_view_with_filter_timing = 1.104

In [48]:
grader.check("q2b")

q2b results: All test cases passed!

### Question 2c:
Given your findings from inspecting the query plans of queries from `Question 2a-b`, answer the following question. Assign the variable `q2c` to a list of all statements which are true.

1. Consider the following statements:
    1. Adding a filter lowered the cost.
    1. Adding a filter increased the cost.
    1. Adding a filter did not change the cost.
    1. Adding a filter increased the execution time.
    1. Adding a filter decreased the execution time.
    1. Adding a filter did not change the execution time.
    1. No statement is true.
    
    
**Note:** Your answer should look like `q2c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q2c
points: 2
-->

In [49]:
q2c = ['A', 'E']

In [50]:
grader.check("q2c")

q2c results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 2d:

Explain your answer based on your knowledge from lectures, and details from the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q2d
manual: true
points: 2
-->

The filter removed 4160 rows which is why it lowered the cost and execution time.

<!-- END QUESTION -->



## Question 3: Join Approaches

In this question, we'll explore different join approaches (Nested Loop Join, Merge Join, Hash Join) and discuss how the query optimizer picks the best approach.

### Question 3a:
Perform an inner join on the `people` and `collegeplaying` tables on the `playerid` column. Project all columns.

In [51]:
%%sql q3a_join <<
SELECT *
FROM people AS p
INNER JOIN collegeplaying AS c
ON p.playerid = c.playerid

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
17350 rows affected.
Returning data to local variable q3a_join


Inspect the query plan for the query you wrote above.

In [52]:
q3a_query_str = "SELECT * FROM people AS p INNER JOIN collegeplaying AS c ON p.playerid = c.playerid"
!psql -h localhost -d baseball -c "explain analyze $q3a_query_str"

                                                        QUERY PLAN                                                         
---------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=861.83..1193.88 rows=17350 width=167) (actual time=7.615..17.018 rows=17350 loops=1)
   Hash Cond: ((c.playerid)::text = (p.playerid)::text)
   ->  Seq Scan on collegeplaying c  (cost=0.00..286.50 rows=17350 width=21) (actual time=0.015..1.271 rows=17350 loops=1)
   ->  Hash  (cost=619.70..619.70 rows=19370 width=146) (actual time=7.428..7.429 rows=19370 loops=1)
         Buckets: 32768  Batches: 1  Memory Usage: 3633kB
         ->  Seq Scan on people p  (cost=0.00..619.70 rows=19370 width=146) (actual time=0.013..1.551 rows=19370 loops=1)
 Planning Time: 1.057 ms
 Execution Time: 17.690 ms
(8 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3a` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3a = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3a
points: 1
-->

In [53]:
q3a = ['C']

In [54]:
grader.check("q3a")

q3a results: All test cases passed!

### Question 3b

Similar to Question 3a, perform an inner join on the `people` and `collegeplaying` tables on the `playerid` column. Project all columns.

In addition, **sort your output by `playerid`.**

In [55]:
%%sql q3b_join <<
SELECT *
FROM people AS p
INNER JOIN collegeplaying AS c
ON p.playerid = c.playerid
ORDER BY p.playerid

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
17350 rows affected.
Returning data to local variable q3b_join


Inspect the query plan for the query you wrote above.

In [56]:
q3b_query_str = "SELECT * FROM people AS p INNER JOIN collegeplaying AS c ON p.playerid = c.playerid ORDER BY p.playerid"
!psql -h localhost -d baseball -c "explain analyze $q3b_query_str"

                                                                         QUERY PLAN                                                                         
------------------------------------------------------------------------------------------------------------------------------------------------------------
 Merge Join  (cost=0.57..1910.20 rows=17350 width=167) (actual time=0.037..18.145 rows=17350 loops=1)
   Merge Cond: ((p.playerid)::text = (c.playerid)::text)
   ->  Index Scan using master_pkey on people p  (cost=0.29..1024.36 rows=19370 width=146) (actual time=0.016..3.830 rows=19368 loops=1)
   ->  Index Only Scan using collegeplaying_pkey on collegeplaying c  (cost=0.29..620.54 rows=17350 width=21) (actual time=0.016..2.794 rows=17350 loops=1)
         Heap Fetches: 0
 Planning Time: 1.020 ms
 Execution Time: 18.827 ms
(7 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3b` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3b = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3b
points: 1
-->

In [57]:
q3b = ['B']

In [58]:
grader.check("q3b")

q3b results: All test cases passed!

### Question 3c
Write a query to retrieve all possible player pair combinations. Project all columns, but **limit to 1000 rows** to ensure your query doesn't take an exorbitant amount of time to run.

**Hint:** You can do this by performing an inner join of the `people` table on itself with an inequality condition.

In [59]:
%%sql
SELECT *
FROM people AS p
INNER JOIN people AS s
ON p.playerid >= s.playerid AND p.playerid <= s.playerid
LIMIT 1000

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1000 rows affected.


playerid,birthyear,birthmonth,birthday,birthcountry,birthstate,birthcity,deathyear,deathmonth,deathday,deathcountry,deathstate,deathcity,namefirst,namelast,namegiven,weight,height,bats,throws,debut,finalgame,retroid,bbrefid,playerid_1,birthyear_1,birthmonth_1,birthday_1,birthcountry_1,birthstate_1,birthcity_1,deathyear_1,deathmonth_1,deathday_1,deathcountry_1,deathstate_1,deathcity_1,namefirst_1,namelast_1,namegiven_1,weight_1,height_1,bats_1,throws_1,debut_1,finalgame_1,retroid_1,bbrefid_1
aardsda01,1981,12,27,USA,CO,Denver,None,None,None,None,None,None,David,Aardsma,David Allan,215,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,aardsda01,1981,12,27,USA,CO,Denver,None,None,None,None,None,None,David,Aardsma,David Allan,215,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
aaronha01,1934,2,5,USA,AL,Mobile,None,None,None,None,None,None,Hank,Aaron,Henry Louis,180,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01,aaronha01,1934,2,5,USA,AL,Mobile,None,None,None,None,None,None,Hank,Aaron,Henry Louis,180,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
aaronto01,1939,8,5,USA,AL,Mobile,1984,8,16,USA,GA,Atlanta,Tommie,Aaron,Tommie Lee,190,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01,aaronto01,1939,8,5,USA,AL,Mobile,1984,8,16,USA,GA,Atlanta,Tommie,Aaron,Tommie Lee,190,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
aasedo01,1954,9,8,USA,CA,Orange,None,None,None,None,None,None,Don,Aase,Donald William,190,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01,aasedo01,1954,9,8,USA,CA,Orange,None,None,None,None,None,None,Don,Aase,Donald William,190,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
abadan01,1972,8,25,USA,FL,Palm Beach,None,None,None,None,None,None,Andy,Abad,Fausto Andres,184,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01,abadan01,1972,8,25,USA,FL,Palm Beach,None,None,None,None,None,None,Andy,Abad,Fausto Andres,184,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01
abadfe01,1985,12,17,D.R.,La Romana,La Romana,None,None,None,None,None,None,Fernando,Abad,Fernando Antonio,220,73.0,L,L,2010-07-28,2017-10-01,abadf001,abadfe01,abadfe01,1985,12,17,D.R.,La Romana,La Romana,None,None,None,None,None,None,Fernando,Abad,Fernando Antonio,220,73.0,L,L,2010-07-28,2017-10-01,abadf001,abadfe01
abadijo01,1850,11,4,USA,PA,Philadelphia,1905,5,17,USA,NJ,Pemberton,John,Abadie,John W.,192,72.0,R,R,1875-04-26,1875-06-10,abadj101,abadijo01,abadijo01,1850,11,4,USA,PA,Philadelphia,1905,5,17,USA,NJ,Pemberton,John,Abadie,John W.,192,72.0,R,R,1875-04-26,1875-06-10,abadj101,abadijo01
abbated01,1877,4,15,USA,PA,Latrobe,1957,1,6,USA,FL,Fort Lauderdale,Ed,Abbaticchio,Edward James,170,71.0,R,R,1897-09-04,1910-09-15,abbae101,abbated01,abbated01,1877,4,15,USA,PA,Latrobe,1957,1,6,USA,FL,Fort Lauderdale,Ed,Abbaticchio,Edward James,170,71.0,R,R,1897-09-04,1910-09-15,abbae101,abbated01
abbeybe01,1869,11,11,USA,VT,Essex,1962,6,11,USA,VT,Colchester,Bert,Abbey,Bert Wood,175,71.0,R,R,1892-06-14,1896-09-23,abbeb101,abbeybe01,abbeybe01,1869,11,11,USA,VT,Essex,1962,6,11,USA,VT,Colchester,Bert,Abbey,Bert Wood,175,71.0,R,R,1892-06-14,1896-09-23,abbeb101,abbeybe01
abbeych01,1866,10,14,USA,NE,Falls City,1926,4,27,USA,CA,San Francisco,Charlie,Abbey,Charles S.,169,68.0,L,L,1893-08-16,1897-08-19,abbec101,abbeych01,abbeych01,1866,10,14,USA,NE,Falls City,1926,4,27,USA,CA,San Francisco,Charlie,Abbey,Charles S.,169,68.0,L,L,1893-08-16,1897-08-19,abbec101,abbeych01


In [60]:
%%sql q3c_join <<
SELECT *
FROM people AS p
INNER JOIN people AS s
ON p.playerid >= s.playerid AND p.playerid <= s.playerid
LIMIT 1000

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1000 rows affected.
Returning data to local variable q3c_join


Inspect the query plan for the query you wrote above.

In [61]:
q3c_query_str = "SELECT * FROM people AS p INNER JOIN people AS s ON p.playerid >= s.playerid AND p.playerid <= s.playerid LIMIT 1000"
!psql -h localhost -d baseball -c "explain analyze $q3c_query_str"

                                                                QUERY PLAN                                                                 
-------------------------------------------------------------------------------------------------------------------------------------------
 Limit  (cost=0.29..30.48 rows=1000 width=292) (actual time=0.043..4.511 rows=1000 loops=1)
   ->  Nested Loop  (cost=0.29..1258723.78 rows=41688544 width=292) (actual time=0.041..4.436 rows=1000 loops=1)
         ->  Seq Scan on people p  (cost=0.00..619.70 rows=19370 width=146) (actual time=0.017..0.107 rows=1000 loops=1)
         ->  Index Scan using master_pkey on people s  (cost=0.29..43.43 rows=2152 width=146) (actual time=0.003..0.003 rows=1 loops=1000)
               Index Cond: (((playerid)::text <= (p.playerid)::text) AND ((playerid)::text >= (p.playerid)::text))
 Planning Time: 0.880 ms
 Execution Time: 4.651 ms
(7 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3c` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3c
points: 1
-->

In [62]:
q3c = ['A']

In [63]:
grader.check("q3c")

q3c results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 3d
Please discuss your findings in the previous parts. In particular, we are interested in hearing why you think the query optimizer chooses the ultimate join approach in each of the above three scenarios. Feel free to discuss the pros and cons of each join approach as well.

If you feel stuck, here are some things to consider: Does a non-equijoin constrain us to certain join approaches? What's an added benefit in regards to the output of merge join?

<!--
BEGIN QUESTION
name: q3d
manual: true
points: 3
-->

Hash join is used to join equi joins. Nested loop join is used for tables that greatly differ in size. Merge join is used to join sorted tables. The optimizer picks the fastest join operator.

<!-- END QUESTION -->



## Question 4: Indexes

### Question 4a:
Write a query that outputs the `playerid` and average `salary` for each player that only batted in 10 games (the number of games in which a player batted can be found in the `g_batting` column of the `appearances` table). Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [70]:
%%sql result_4a <<
SELECT a.playerid, AVG(s.salary)
FROM appearances AS a
NATURAL JOIN salaries AS s
WHERE a.g_batting = 10
GROUP BY a.playerid

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
134 rows affected.
Returning data to local variable result_4a


In [71]:
# Do not delete/edit this cell
result_4a.DataFrame().sort_values('playerid').to_csv('results/result_4a.csv', index=False)

In [72]:
grader.check("q4ai")

q4ai results: All test cases passed!

Inspect the query plan and record the execution time and cost.

In [73]:
result_4a_query_str = "SELECT a.playerid, AVG(s.salary) FROM appearances AS a NATURAL JOIN salaries AS s WHERE a.g_batting = 10 GROUP BY a.playerid"
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                   QUERY PLAN                                                                                   
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=3636.18..3636.21 rows=1 width=17) (actual time=50.701..50.867 rows=134 loops=1)
   Group Key: a.playerid
   ->  Sort  (cost=3636.18..3636.19 rows=1 width=17) (actual time=50.684..50.706 rows=138 loops=1)
         Sort Key: a.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=2899.40..3636.17 rows=1 width=17) (actual time=30.151..50.325 rows=138 loops=1)
               Hash Cond: ((s.yearid = a.yearid) AND ((s.teamid)::text = (a.teamid)::text) AND ((s.lgid)::text = (a.lgid)::text) AND ((s.playerid)::text = (a.playerid)::text))
               ->  Seq Scan on salaries s  (cost=0.00..45

In [74]:
result_4a_cost = 3636.21
result_4a_timing = 51.058

In [75]:
grader.check("q4aii")

q4aii results: All test cases passed!

### Question 4b:
Add an index with name `g_batting_idx` on the `g_batting` column of the `appearances` table.

In [86]:
%%sql result_4b <<
CREATE INDEX g_batting_idx
ON appearances (g_batting)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
(psycopg2.errors.DuplicateTable) relation "g_batting_idx" already exists

[SQL: CREATE INDEX g_batting_idx
ON appearances (g_batting)]
(Background on this error at: https://sqlalche.me/e/14/f405)


Now, re-inspect the query plan of the query from `Question 4a` and record its execution time and cost.

In [98]:
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                   QUERY PLAN                                                                                   
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=3636.18..3636.21 rows=1 width=17) (actual time=22.629..22.700 rows=134 loops=1)
   Group Key: a.playerid
   ->  Sort  (cost=3636.18..3636.19 rows=1 width=17) (actual time=22.621..22.629 rows=138 loops=1)
         Sort Key: a.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=2899.40..3636.17 rows=1 width=17) (actual time=14.527..22.457 rows=138 loops=1)
               Hash Cond: ((s.yearid = a.yearid) AND ((s.teamid)::text = (a.teamid)::text) AND ((s.lgid)::text = (a.lgid)::text) AND ((s.playerid)::text = (a.playerid)::text))
               ->  Seq Scan on salaries s  (cost=0.00..45

In [88]:
result_4b_cost = 2367.80
result_4b_timing = 20.227

In [89]:
grader.check("q4b")

q4b results: All test cases passed!

In the following question, we will explore adding a different index and evaluating the query from `Question 4a`. To avoid any interference by the `g_batting_idx` index, **drop the index before moving onto the next question.**

In [93]:
%%sql 
DROP INDEX g_batting_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 4c:
Write a query to add an index with name `salary_idx` on the `salary` column of the `salaries` table.

In [94]:
%%sql result_4c <<
CREATE INDEX salary_idx
ON salaries (salary)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4c


Re-inspect the query plan of the query from the `Question 4a` and record its execution time and cost.

In [95]:
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                   QUERY PLAN                                                                                   
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=3636.18..3636.21 rows=1 width=17) (actual time=22.026..22.096 rows=134 loops=1)
   Group Key: a.playerid
   ->  Sort  (cost=3636.18..3636.19 rows=1 width=17) (actual time=22.018..22.025 rows=138 loops=1)
         Sort Key: a.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=2899.40..3636.17 rows=1 width=17) (actual time=14.238..21.856 rows=138 loops=1)
               Hash Cond: ((s.yearid = a.yearid) AND ((s.teamid)::text = (a.teamid)::text) AND ((s.lgid)::text = (a.lgid)::text) AND ((s.playerid)::text = (a.playerid)::text))
               ->  Seq Scan on salaries s  (cost=0.00..45

In [96]:
result_4c_cost = 3636.21
result_4c_timing = 22.223

In [97]:
grader.check("q4c")

q4c results: All test cases passed!

### Question 4d:
#### Question 4di:
Given your findings from inspecting the query plans with no indexes, an index on `g_batting`, and an index on `salary`, answer the following question. Assign the variable `q4d_part1` to a list of all options which are true for the proposed statement.

1. Consider the following statements:
    1. Adding the `g_batting` index did not have a significant impact on the query execution time and cost.
    1. Adding the `g_batting` index did have a significant impact on the query execution time, but not the cost.
    1. Adding the `g_batting` index did have a significant impact on the query cost, but not the execution time.
    1. Adding the `g_batting` index did have a significant impact on the query cost and execution time.
    1. Adding the `salary` index did not have a significant impact on the query execution time and cost.
    1. Adding the `salary` index did have a significant impact on the query execution time, but not the cost.
    1. Adding the `salary` index did have a significant impact on the query cost, but not the execution time.
    1. Adding the `salary` index did have a significant impact on the query cost and execution time.

**Note:** Your answer should be formatted as follows: `q4d_part1 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q4di
points: 2
-->

In [212]:
q4d_part1 = ['D', 'E']

In [213]:
grader.check("q4di")

q4di results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 4dii:

Explain your answer based on your knowledge from lectures, and details from inspecting the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q4dii
manual: true
points: 2
-->

Adding the g_batting index had a significant impact because of the WHERE condition needs to find the data while the salary index did not have an impact because AVG does not make use of an index.

<!-- END QUESTION -->



In the following question, we will further explore the impact of indexes on query performance. To avoid any interference by the `salary_idx` index, please drop the index before moving onto the next question.

In [101]:
%%sql 
DROP INDEX salary_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 4e:
#### Question 4ei:
Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games and batted in 10 games (the number of games in which a player played can be found in the `g_all` column of the `appearances` table). Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [105]:
%%sql result_4ei <<
SELECT a.playerid, a.yearid, s.salary
FROM appearances AS a
NATURAL JOIN salaries AS s
WHERE g_batting = 10 AND g_all = 10

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
120 rows affected.
Returning data to local variable result_4ei


In [107]:
# Do not delete/edit this cell
result_4ei.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_4ei.csv', index=False)

In [108]:
grader.check("q4ei_part1")

q4ei_part1 results: All test cases passed!

Inspect the query plan and record the execution time and cost.

In [109]:
result_4ei_query_str = "SELECT a.playerid, a.yearid, s.salary FROM appearances AS a NATURAL JOIN salaries AS s WHERE g_batting = 10 AND g_all = 10"
!psql -h localhost -d baseball -c "explain analyze $result_4ei_query_str"

                                                                            QUERY PLAN                                                                             
-------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=0.29..3296.09 rows=1 width=21) (actual time=9.183..16.481 rows=120 loops=1)
   ->  Seq Scan on appearances a  (cost=0.00..3133.84 rows=20 width=20) (actual time=0.021..13.063 rows=1289 loops=1)
         Filter: ((g_batting = 10) AND (g_all = 10))
         Rows Removed by Filter: 102967
   ->  Index Scan using salaries_pkey on salaries s  (cost=0.29..8.11 rows=1 width=28) (actual time=0.002..0.002 rows=0 loops=1289)
         Index Cond: ((yearid = a.yearid) AND ((teamid)::text = (a.teamid)::text) AND ((lgid)::text = (a.lgid)::text) AND ((playerid)::text = (a.playerid)::text))
 Planning Time: 2.467 ms
 Execution Time: 16.551 ms
(8 rows)



In [110]:
result_4ei_cost = 3296.09
result_4ei_timing = 16.551

In [111]:
grader.check("q4ei_part2")

q4ei_part2 results: All test cases passed!

#### Question 4eii:
Now, let's see the impact of adding an index on the `g_batting` column. Create an index on the `g_batting` column. Re-inspect the query from `Question 4ei` and record the execution time and cost.

In [112]:
%%sql result_4eii << 
CREATE INDEX g_batting_idx
ON appearances (g_batting)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4eii


In [113]:
!psql -h localhost -d baseball -c "explain analyze $result_4ei_query_str"

                                                                            QUERY PLAN                                                                             
-------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=18.41..1770.00 rows=1 width=21) (actual time=2.782..5.538 rows=120 loops=1)
   ->  Bitmap Heap Scan on appearances a  (cost=18.12..1607.75 rows=20 width=20) (actual time=0.315..2.469 rows=1289 loops=1)
         Recheck Cond: (g_batting = 10)
         Filter: (g_all = 10)
         Rows Removed by Filter: 58
         Heap Blocks: exact=899
         ->  Bitmap Index Scan on g_batting_idx  (cost=0.00..18.12 rows=1310 width=0) (actual time=0.181..0.181 rows=1347 loops=1)
               Index Cond: (g_batting = 10)
   ->  Index Scan using salaries_pkey on salaries s  (cost=0.29..8.11 rows=1 width=28) (actual time=0.002..0.002 rows=0 loops=1289)
     

In [114]:
result_4eii_with_index_cost = 1770
result_4eii_with_index_timing = 5.63

In [115]:
grader.check("q4eii")

q4eii results: All test cases passed!

In the following question, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `g_batting` column, drop the index before moving onto the next question.

In [117]:
%%sql 
DROP INDEX g_batting_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

#### Question 4eiii:
Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games __or__ batted in 10 games. Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [118]:
%%sql result_4eiii <<
SELECT a.playerid, a.yearid, s.salary
FROM appearances AS a
NATURAL JOIN salaries AS s
WHERE a.g_all = 10 OR a.g_batting = 10

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
216 rows affected.
Returning data to local variable result_4eiii


In [119]:
# Do not delete/edit this cell
result_4eiii.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_4eiii.csv', index=False)

In [120]:
grader.check("q4eiii_part1")

q4eiii_part1 results: All test cases passed!

In [121]:
result_4eiii_query_str = "SELECT a.playerid, a.yearid, s.salary FROM appearances AS a NATURAL JOIN salaries AS s WHERE a.g_all = 10 OR a.g_batting = 10"
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                             QUERY PLAN                                                                             
--------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=3190.86..3927.63 rows=1 width=21) (actual time=16.086..30.101 rows=216 loops=1)
   Hash Cond: ((s.yearid = a.yearid) AND ((s.teamid)::text = (a.teamid)::text) AND ((s.lgid)::text = (a.lgid)::text) AND ((s.playerid)::text = (a.playerid)::text))
   ->  Seq Scan on salaries s  (cost=0.00..459.28 rows=26428 width=28) (actual time=0.019..3.201 rows=26428 loops=1)
   ->  Hash  (cost=3133.84..3133.84 rows=2851 width=20) (actual time=15.764..15.766 rows=1655 loops=1)
         Buckets: 4096  Batches: 1  Memory Usage: 121kB
         ->  Seq Scan on appearances a  (cost=0.00..3133.84 rows=2851 width=20) (actual time=0.015..15.026 rows=1655 loops=1)
      

In [122]:
result_4eiii_cost = 3927.63
result_4eiii_timing = 30.186

In [123]:
grader.check("q4eiii_part2")

q4eiii_part2 results: All test cases passed!

#### Question 4eiv
Now, let's see the impact of adding an index on `g_batting` column will have on the query. Re-create the index and re-inspect the query from `Question 4eiii` and record the execution time and cost.

In [124]:
%%sql result_4eiv << 
CREATE INDEX g_batting_idx
ON appearances (g_batting)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4eiv


In [125]:
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                             QUERY PLAN                                                                             
--------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=3190.86..3927.63 rows=1 width=21) (actual time=28.809..44.810 rows=216 loops=1)
   Hash Cond: ((s.yearid = a.yearid) AND ((s.teamid)::text = (a.teamid)::text) AND ((s.lgid)::text = (a.lgid)::text) AND ((s.playerid)::text = (a.playerid)::text))
   ->  Seq Scan on salaries s  (cost=0.00..459.28 rows=26428 width=28) (actual time=0.028..3.561 rows=26428 loops=1)
   ->  Hash  (cost=3133.84..3133.84 rows=2851 width=20) (actual time=28.425..28.427 rows=1655 loops=1)
         Buckets: 4096  Batches: 1  Memory Usage: 121kB
         ->  Seq Scan on appearances a  (cost=0.00..3133.84 rows=2851 width=20) (actual time=0.025..27.083 rows=1655 loops=1)
      

In [126]:
result_4eiv_with_index_cost = 3927.63
result_4eiv_with_index_timing = 44.930

In [127]:
grader.check("q4eiv")

q4eiv results: All test cases passed!

In the following question, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `g_batting` column, drop the index before moving onto the next question.

In [128]:
%%sql 
DROP INDEX g_batting_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

#### Question 4ev:
Now, create a multiple column index on `g_batting` and `g_all` called `g_batting_g_all_idx` and record the query execution time and cost.

In [129]:
%%sql result_4v <<
CREATE INDEX g_batting_g_all_idx
ON appearances (g_batting, g_all)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4v


In [130]:
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                             QUERY PLAN                                                                             
--------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=2883.30..3620.08 rows=1 width=21) (actual time=4.391..12.193 rows=216 loops=1)
   Hash Cond: ((s.yearid = a.yearid) AND ((s.teamid)::text = (a.teamid)::text) AND ((s.lgid)::text = (a.lgid)::text) AND ((s.playerid)::text = (a.playerid)::text))
   ->  Seq Scan on salaries s  (cost=0.00..459.28 rows=26428 width=28) (actual time=0.021..1.890 rows=26428 loops=1)
   ->  Hash  (cost=2826.28..2826.28 rows=2851 width=20) (actual time=4.182..4.184 rows=1655 loops=1)
         Buckets: 4096  Batches: 1  Memory Usage: 121kB
         ->  Bitmap Heap Scan on appearances a  (cost=1181.76..2826.28 rows=2851 width=20) (actual time=1.383..3.698 rows=1655 loops=1)

In [131]:
result_4ev_multiple_col_index_cost = 3620.08
result_4ev_multiple_col_index_timing = 12.294

In [132]:
grader.check("q4ev")

q4ev results: All test cases passed!

#### Question 4evi:
Given your findings from inspecting the query plans from `Question 4e`, answer the following question. Assign the variable `q4e_part6` to a list of all statements that are true.

1. Consider the following statements:
    1. Adding an index on a column used in an AND predicate will reduce the query time, but not the execution cost.
    1. Adding an index on a column used in an AND predicate will reduce the query cost, but not the execution time.
    1. Adding an index on a column used in an AND predicate will reduce the query cost and the execution time.
    1. Adding an index on a column used in an OR predicate will reduce the query time, but not the execution cost.
    1. Adding an index on a column used in an OR predicate will reduce the query cost, but not the execution time.
    1. Adding an index on a column used in an OR predicate will reduce the query cost and the execution time.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query time, but not the execution cost.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query cost, but not the execution time.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query cost and the execution time.

**Note:** Your answer should be formatted as follows: `q4e_part6 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q4evi
points: 2
-->

In [133]:
q4e_part6 = ['C', 'I']

In [134]:
grader.check("q4evi")

q4evi results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 4evii

Explain your answer to the previous part based on your knowledge from lectures, and details of the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q4evii
manual: true
points: 2
-->

AND predicates are more exclusive than OR predicates so the index is used to filter but with the OR predicate the table needs to be looped through multiple times.

<!-- END QUESTION -->



## Question 5: More Indexes
### Question 5a:
Write 2 queries: one that finds the minimum salary from the salary table and one that finds the average. Inspect the queries' query plans and record their execution times and costs.

In [141]:
%%sql result_5a_min << 
SELECT MIN(salary)
FROM salaries

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1 rows affected.
Returning data to local variable result_5a_min


In [142]:
result_5a_min_query_str = "SELECT MIN(salary), AVG(salary) FROM salaries"
!psql -h localhost -d baseball -c "explain analyze $result_5a_min_query_str"

                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=591.42..591.43 rows=1 width=16) (actual time=5.012..5.013 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.050..1.821 rows=26428 loops=1)
 Planning Time: 0.341 ms
 Execution Time: 5.074 ms
(4 rows)



In [143]:
result_5a_min_query_cost = 591.43
result_5a_min_query_timing = 5.074

In [144]:
grader.check("q5ai")

q5ai results: All test cases passed!

In [145]:
%%sql result_5a_avg <<
SELECT AVG(salary)
FROM salaries

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1 rows affected.
Returning data to local variable result_5a_avg


In [147]:
result_5a_avg_query_str = "SELECT AVG(salary) FROM salaries"
!psql -h localhost -d baseball -c "explain analyze $result_5a_avg_query_str"

                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=525.35..525.36 rows=1 width=8) (actual time=4.869..4.870 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.025..1.855 rows=26428 loops=1)
 Planning Time: 0.445 ms
 Execution Time: 4.937 ms
(4 rows)



In [148]:
result_5a_avg_query_cost = 525.36
result_5a_avg_query_timing = 4.937

In [149]:
grader.check("q5aii")

q5aii results: All test cases passed!

### Question 5b:
Create an index on the `salary` column in the `salaries` table and re-inspect the query plans from the previous part and record the respective execution time and cost.

In [150]:
%%sql 
CREATE INDEX salary_idx
ON salaries (salary)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

In [151]:
!psql -h localhost -d baseball -c "explain analyze $result_5a_min_query_str"

                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=591.42..591.43 rows=1 width=16) (actual time=4.917..4.918 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.020..1.729 rows=26428 loops=1)
 Planning Time: 0.555 ms
 Execution Time: 4.991 ms
(4 rows)



In [152]:
result_5b_min_query_cost = 591.43
result_5b_min_query_timing = 4.991

In [153]:
grader.check("q5bi")

q5bi results: All test cases passed!

In [154]:
!psql -h localhost -d baseball -c "explain analyze $result_5a_avg_query_str"

                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=525.35..525.36 rows=1 width=8) (actual time=5.042..5.043 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.017..1.970 rows=26428 loops=1)
 Planning Time: 0.471 ms
 Execution Time: 5.116 ms
(4 rows)



In [155]:
result_5b_avg_query_cost = 525.36
result_5b_avg_query_timing = 5.116

In [156]:
grader.check("q5bii")

q5bii results: All test cases passed!

In the following questions, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `salary` column, drop the index before moving onto the next question.

In [157]:
%%sql 
DROP INDEX salary_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 5c:
Given your findings from `Question 5a-b`, answer the following question. Assign the variable `q5c` to the correct answer choice.

1. Which of the following statements is true?
    1. An index on the column being aggregated in a query will always provide a performance enhancement.
    1. A query finding the MIN(salary) will always benefit from an index on salary, but a query finding MAX(salary) will not.
    1. A query finding the COUNT(salary) will always benefit from an index on salary, but a query finding AVG(salary) will not.
    1. Queries finding the MIN(salary) or MAX(salary) will always benefit from an index on salary, but queries finding AVG(salary) or COUNT(salary) will not.

*Note:* Your answer should be formatted as follows: `q5c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q5c
points: 1
-->

In [160]:
q5c = ['D']

In [161]:
grader.check("q5c")

q5c results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 5d:

Explain your answer to the previous part based on your knowledge from lectures, and your inspection of the query plans.

<!--
BEGIN QUESTION
name: q5d
manual: true
points: 2
-->

Only order specific aggregrates like MIN and MAX benefit from index, while AVG and COUNT still need to go through the whole table.

<!-- END QUESTION -->



## Question 6: Clustered Indexes
In this question, we will inspect the impact that clustering our data on an index can have on a query's performance.

### Question 6a
Write a query that finds the `playerid`, `yearid`, `teamid`, and `ab` for all players whose `ab` was above 500. Inspect the query plan and record the execution time and cost.

In [167]:
%%sql result_6a <<
SELECT playerid, yearid, teamid, ab
FROM batting
WHERE ab > 500

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
8839 rows affected.
Returning data to local variable result_6a


In [168]:
# Do not delete/edit this cell
result_6a.DataFrame().sort_values(['playerid', 'yearid', 'teamid']).iloc[:1000].to_csv('results/result_6a.csv', index=False)

In [169]:
grader.check("q6ai")

q6ai results: All test cases passed!

In [170]:
result_6a_query_str = "SELECT playerid, yearid, teamid, ab FROM batting WHERE ab > 500"
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                 QUERY PLAN                                                 
------------------------------------------------------------------------------------------------------------
 Seq Scan on batting  (cost=0.00..2884.05 rows=8926 width=21) (actual time=0.299..13.669 rows=8839 loops=1)
   Filter: (ab > 500)
   Rows Removed by Filter: 95485
 Planning Time: 0.368 ms
 Execution Time: 14.043 ms
(5 rows)



In [171]:
result_6a_cost = 2884.05
result_6a_timing = 14.043

In [172]:
grader.check("q6aii")

q6aii results: All test cases passed!

### Question 6b:
Cluster the `batting` table on its primary key (hint: use `\di` to find out what name of the primary key is). We are able to directly cluster on the primary key (without first creating a separate index) because Postgres automatically creates an index for it. Re-inspect the query plan for the query from `Question 6a` and record the execution time and cost.

In [183]:
%%sql 
CLUSTER batting USING batting_pkey

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

In [184]:
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                 QUERY PLAN                                                 
------------------------------------------------------------------------------------------------------------
 Seq Scan on batting  (cost=0.00..2878.05 rows=8926 width=21) (actual time=0.016..14.829 rows=8839 loops=1)
   Filter: (ab > 500)
   Rows Removed by Filter: 95485
 Planning Time: 0.468 ms
 Execution Time: 15.194 ms
(5 rows)



In [185]:
result_6b_cost = 2878.05
result_6b_timing = 15.194

In [186]:
grader.check("q6b")

q6b results: All test cases passed!

### Question 6c
Now, let's try clustering the table based on another index. Create an index on `ab` column called `ab_idx` in the `batting` table AND cluster the `batting` table with this new index. Now, re-inspect the query plan and record the execution time and cost.

In [194]:
%%sql 
CREATE INDEX ab_idx
ON batting (ab);

CLUSTER batting USING ab_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.


[]

In [195]:
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                      QUERY PLAN                                                      
----------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on batting  (cost=101.47..1788.04 rows=8926 width=21) (actual time=0.396..2.151 rows=8839 loops=1)
   Recheck Cond: (ab > 500)
   Heap Blocks: exact=135
   ->  Bitmap Index Scan on ab_idx  (cost=0.00..99.24 rows=8926 width=0) (actual time=0.362..0.362 rows=8839 loops=1)
         Index Cond: (ab > 500)
 Planning Time: 0.573 ms
 Execution Time: 2.526 ms
(7 rows)



In [196]:
result_6c_cost = 1788.04
result_6c_timing = 2.526

In [197]:
grader.check("q6c")

q6c results: All test cases passed!

### Question 6d
Given your findings from inspecting the query plans from `Question 6a-c`, answer the following question. Assign the variable `q6d` to a list of all statements that are true.

1. Consider the following statements:
    1. Clustering based on the `ab_idx` decreased the cost of the query.
    1. Clustering based on the `ab_idx` increased the cost of the query.
    1. Clustering based on the `ab_idx` increased the execution time of the query.
    1. Clustering based on the `ab_idx` decreased the execution time of the query.
    1. Clustering based on the `batting_pkey` decreased the cost of the query.
    1. Clustering based on the `batting_pkey` increased the cost of the query.
    1. Clustering based on the `batting_pkey` increased the execution time of the query.
    1. Clustering based on the `batting_pkey` decreased the execution time of the query.
    1. None of the above
    
**Note:** Your answer should be formatted as follows: `q6d = ['A', 'B']`

<!--
BEGIN QUESTION
name: q6d
points: 2
-->

In [198]:
q6d = ['A', 'D', 'E', 'G']

In [199]:
grader.check("q6d")

q6d results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 6e:

Explain your answer to the previous part based on your knowledge from lectures, and your inspection of the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q6e
manual: true
points: 2
-->

Clustering on the ab_idx greatly reduced the cost and execution time of the query, where as clustering on the batting_pkey only reduced the cost by a little and had a greater execution time.

<!-- END QUESTION -->



## Question 7: Cost of Index Management
Until now, we have seen the positive potential impact that indexes can have on query performance, but remember in real world technologies/applications, we will be routinely receiving new data (and in large quantities) which would trigger regular updates to our tables. In this section, we will dive into the cost of managing the indexes that we create.

### Question 7a:
Record the time it takes to insert 300,000 rows into the `salaries` table when no additional index is configured.

Run the following cell to setup a column to track which rows we added as part of these inserts.

In [200]:
%sql ALTER TABLE salaries ADD added boolean DEFAULT False;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

Next, run the provided update script and record the **wall time**.

**NOTE:** Running the below cell multiple times may result in an error, unless you first delete the rows with the cell given at the end of this subpart.

In [201]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
CPU times: user 2.59 ms, sys: 2.53 ms, total: 5.12 ms
Wall time: 3.56 s


[]

In [202]:
result_7a_timing = 3.56

In [203]:
grader.check("q7a")

q7a results: All test cases passed!

Before adding an index to the salaries table and re-timing the updates, delete all the rows that were added to the table from the update script.

In [204]:
%%sql
DELETE FROM salaries
WHERE added = 'true';

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
300000 rows affected.


[]

### Question 7b:
Now, create an index on the `salary` column and record the **wall time** after executing the update script.

In [205]:
%%sql 
CREATE INDEX salary_idx
ON salaries (salary)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

**NOTE:** Running the below cell multiple times may result in an error, unless you first delete the rows with the cell given at the end of last subpart.

In [206]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
CPU times: user 2.72 ms, sys: 1.9 ms, total: 4.62 ms
Wall time: 5.81 s


[]

In [207]:
result_7b_timing = 5.81

In [208]:
grader.check("q7b")

q7b results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 7c:
What difference did you notice when you added an index into the salaries table and re-timed the update? Why do you think it happened?

<!--
BEGIN QUESTION
name: q7c
manual: true
points: 2
-->

The wall time increased when I added an index which I think is due to the added index being less suitable for the query or it having another index to compare.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## Question 8: Takeaway

In this project, we explored how the database system optimizes query execution and how users can futher tune the performance of their queries.

Familiarizing yourself with these optimization and tuning methods will make you a better data engineer. In this question, we'll ask you to recall and summarize these concepts. Who knows? Maybe one day it will help you during an interview or on a project.

In the following answer cell,
1. Name 3 methods you learned in this project. The method can be either the optimization done by the database system, or the fine tuning done by the user.
2. For each method, summarize how and why it can optimize query performance. Feel free to discuss any drawbacks, if applicable.

<!--
BEGIN QUESTION
name: q8
manual: true
points: 6
-->

1. filter, index, and cluster
2. - Removing unneeded rows lowers cost and execution times as there are less rows to work with
   - Indexing creates pointers so finding data is much faster
   - Clustering physically reorders data on a disk which, similar to indexing, helps data be found faster.

<!-- END QUESTION -->



## Congratulations! You have finished Project 2.

Run the following cell to zip and download the results of your queries. You will also need to run the export cell at the end of the notebook.

**Please save your notebook before exporting (this is a good time to do it!)** Otherwise, we may not be able to register your written responses.

**For submission on Gradescope, you will need to submit BOTH the proj2.zip file genreated by the export cell and the results.zip file generated by the following cell.**

**Common submission issues:** You MUST submit the generated zip files (not folders) to the autograder. However, Safari is known to automatically unzip files upon downloading. You can fix this by going into Safari preferences, and deselect the box with the text "Open safe files after downloading" under the "General" tab. If you experience issues with downloading via clicking on the link, you can also navigate to the project 2 directory within JupyterHub (remove `proj2.ipynb` from the url), and manually download the generated zip files. Please post on Ed if you encounter any other submission issues.

In [214]:
from IPython.display import display, FileLink

!zip -r results.zip results
results_file = FileLink('./results.zip', result_html_prefix="Click here to download: ")
display(results_file)

updating: results/ (stored 0%)
updating: results/result_1a.csv (deflated 53%)
updating: results/result_1c.csv (deflated 64%)
updating: results/result_1di_view.csv (deflated 37%)
updating: results/result_1di_no_view.csv (deflated 37%)
updating: results/result_1ei.csv (deflated 64%)
updating: results/result_1eii.csv (deflated 37%)
updating: results/result_4a.csv (deflated 59%)
updating: results/result_4ei.csv (deflated 62%)
updating: results/result_4eiii.csv (deflated 64%)
updating: results/result_6a.csv (deflated 72%)


/home/jovyan/fa22/proj/proj2/results.zip

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [215]:
grader.check_all()

q0 results: All test cases passed!

q1a results: All test cases passed!

q1bi results: All test cases passed!

q1bii results: All test cases passed!

q1c results: All test cases passed!

q1di results: All test cases passed!

q1dii results: All test cases passed!

q1ei results: All test cases passed!

q1eii results: All test cases passed!

q1eiii results: All test cases passed!

q1eiv results: All test cases passed!

q2a results: All test cases passed!

q2b results: All test cases passed!

q2c results: All test cases passed!

q3a results: All test cases passed!

q3b results: All test cases passed!

q3c results: All test cases passed!

q4ai results: All test cases passed!

q4aii results: All test cases passed!

q4b results: All test cases passed!

q4c results: All test cases passed!

q4di results: All test cases passed!

q4ei_part1 results: All test cases passed!

q4ei_part2 results: All test cases passed!

q4eii results: All test cases passed!

q4eiii_part1 results: All test cases passe

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [216]:
# Save your notebook first, then run this cell to export your submission.
grader.export()